In [1]:
#!pip install razdel

In [2]:
#!pip install pymorphy2

In [3]:
#!pip install pyLDAvis 

# ДЗ №2

1) Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

2) Повторить п.2, но используя уже не медиану, а max 

3) (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

4) Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

5) Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [4]:
import pandas as pd
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
import nltk

from razdel import tokenize # https://github.com/natasha/razdel
import pymorphy2  

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from gensim.models import LdaModel
from gensim.test.utils import datapath
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
news = pd.read_csv("C:/Users/User/Desktop/Машинное обучение в бизнесе/2/Lection2/articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [6]:
#Загрузим пользователей и списки последних прочитанных новостей
users = pd.read_csv('C:/Users/User/Desktop/Машинное обучение в бизнесе/2/Lection2/users_articles.csv')
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [7]:
# Нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей:

stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

morph = pymorphy2.MorphAnalyzer()

151


In [8]:
with open('C:/Users/User/Desktop/Машинное обучение в бизнесе/2/Lection2/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [9]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str): #isinstance функция для проверки принодлежности обьекта конкретному типу.
        text = str(text)
    
    text = text.lower() #меняем гегистр всех знаков
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [10]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Users\User\AppData\Local\Temp\ipykernel_3048\4233062323.py:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 24.5 s


In [11]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 4min 10s


In [12]:
# Обучение модели:

#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [13]:
# Что такое common_dictionary и как он выглядит

#common_dictionary[6]
# 'ватутин'
# Все просто - это словарь наших слов

In [14]:
# Запускаем обучение:
#%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

In [15]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [16]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(7, 0.11914193),
 (8, 0.26348686),
 (11, 0.3613096),
 (12, 0.064121455),
 (16, 0.17399944)]

In [17]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: год научный статья который время тело университет
topic_1: космос доллар виза устойчивый атмосферный армения визовый
topic_2: взрыв напомнить помещение миллиард мышь сенатор снятие
topic_3: банк завод рынок высота карта продажа продолжительность
topic_4: доля строительство кость управление адмирал квадратный русь
topic_5: год белый дом конкурс налог министерство вицепремьер
topic_6: исследование год который это человек рубль также
topic_7: год это который мочь млрд рост система
topic_8: nn это стать первый который год весь
topic_9: смерть женщина остров год превысить великобритания британский
topic_10: бизнесмен сестра сопровождать ресторан савченко разойтись непрерывный
topic_11: млн год спрос составить доход стоимость физика
topic_12: это который год свой человек мочь россия
topic_13: студент золото золотой соль номинация браун узбекистан
topic_14: мужчина убийство суд дело преступление уголовный египет
topic_15: год военный наука армия который участок торговый
topic_16: это

In [18]:
#пишем функцию, которая будет возвращать векторное представление новости:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [19]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.000000,0.0,0.0,0.0,0.029299,0.274081,0.064821,...,0.0,0.000000,0.0,0.360688,0.0,0.141255,0.122901,0.0,0.0,0.0
1,4896,0.0,0.0,0.483812,0.0,0.0,0.0,0.000000,0.000000,0.421756,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
2,4897,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.119323,0.263386,...,0.0,0.174070,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
3,4898,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.370778,...,0.0,0.240083,0.0,0.173755,0.0,0.000000,0.000000,0.0,0.0,0.0
4,4899,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.185721,...,0.0,0.000000,0.0,0.000000,0.0,0.788683,0.000000,0.0,0.0,0.0


In [20]:
#векторные представления пользователей:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [21]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [22]:
doc_dict[293622]

array([0.        , 0.        , 0.01809048, 0.        , 0.        ,
       0.        , 0.08787991, 0.02973564, 0.3117471 , 0.06250432,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03155826, 0.        , 0.08178458, 0.07445233,
       0.1140935 , 0.14587134, 0.0337828 , 0.        , 0.        ])

# ДЗ выполнение:

In [23]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [25]:
#получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче:
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_mean.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.024755,0.000000,0.01590,0.020017,0.0,0.037820,0.135992,0.038521,0.155976,...,0.072400,0.009042,0.000000,0.033227,0.027163,0.021490,0.205003,0.067852,0.016372,0.000000
1,u108690,0.029158,0.000000,0.00000,0.004120,0.0,0.021783,0.089308,0.120790,0.027818,...,0.000000,0.037507,0.000000,0.040351,0.001676,0.046510,0.201567,0.107216,0.021533,0.019292
2,u108339,0.082629,0.003023,0.01202,0.000000,0.0,0.046814,0.156236,0.052618,0.034328,...,0.047266,0.026835,0.013799,0.025329,0.000000,0.069121,0.224069,0.019933,0.005502,0.017396


In [26]:
user_embeddings_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings_median.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_median['uid'] = users['uid'].values
user_embeddings_median = user_embeddings_median[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_median.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.000000,0.0,0.009045,0.0,0.0,0.000000,0.155515,0.000000,0.169332,...,0.032972,0.000000,0.0,0.021153,0.016677,0.000000,0.213408,0.016891,0.000000,0.000000
1,u108690,0.022248,0.0,0.000000,0.0,0.0,0.008782,0.077553,0.086023,0.000000,...,0.000000,0.027988,0.0,0.030560,0.000000,0.000000,0.179803,0.082782,0.013523,0.018564
2,u108339,0.057788,0.0,0.007854,0.0,0.0,0.039509,0.157848,0.008763,0.036962,...,0.043456,0.008660,0.0,0.000000,0.000000,0.039768,0.241592,0.011089,0.000000,0.008589


In [27]:
user_embeddings_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings_max.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_max['uid'] = users['uid'].values
user_embeddings_max = user_embeddings_max[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_max.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.148532,0.00000,0.041426,0.120103,0.0,0.226918,0.246467,0.201393,0.311747,...,0.292169,0.031558,0.000000,0.081785,0.074452,0.114094,0.352725,0.283946,0.098234,0.000000
1,u108690,0.070447,0.00000,0.000000,0.024723,0.0,0.066690,0.227178,0.273076,0.138516,...,0.000000,0.094129,0.000000,0.091197,0.010058,0.194875,0.343468,0.234713,0.056399,0.044081
2,u108339,0.232033,0.01814,0.039699,0.000000,0.0,0.088160,0.261277,0.223659,0.069054,...,0.091456,0.110265,0.082797,0.083728,0.000000,0.219093,0.286423,0.070731,0.033010,0.060413


In [28]:
# попробуем обучить модель. Загрузим нашу разметку:
target = pd.read_csv('C:/Users/User/Desktop/Машинное обучение в бизнесе/2/Lection2/users_churn.csv')
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [29]:
X_mean = pd.merge(user_embeddings_mean, target, 'left')

X_median = pd.merge(user_embeddings_median, target, 'left')

X_max = pd.merge(user_embeddings_max, target, 'left')

In [30]:
#разделим данные на train/test
X_mean_train, X_mean_test, y_mean_train, y_mean_test = train_test_split(X_mean[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_mean['churn'], random_state=0)

In [31]:
X_median_train, X_median_test, y_median_train, y_median_test = train_test_split(X_median[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_median['churn'], random_state=0)

In [32]:
X_max_train, X_max_test, y_max_train, y_max_test = train_test_split(X_max[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_max['churn'], random_state=0)

In [33]:
logreg_mean = LogisticRegression()
#обучим 
logreg_mean.fit(X_mean_train, y_mean_train)

LogisticRegression()

In [34]:
logreg_median = LogisticRegression()
#обучим 
logreg_median.fit(X_median_train, y_median_train)

LogisticRegression()

In [35]:
logreg_max = LogisticRegression()
#обучим 
logreg_max.fit(X_max_train, y_max_train)

LogisticRegression()

In [42]:
#наши прогнозы для тестовой выборки
preds_mean = logreg_mean.predict_proba(X_mean_test)[:, 1]
preds_mean[:10]

array([0.2664149 , 0.06584747, 0.47099016, 0.27531161, 0.1126306 ,
       0.05405242, 0.14327924, 0.00827505, 0.1671039 , 0.2837156 ])

In [44]:
#наши прогнозы для тестовой выборки
preds_median = logreg_median.predict_proba(X_median_test)[:, 1]
preds_median[:10]

array([0.29710015, 0.07481311, 0.5028971 , 0.24112394, 0.16238166,
       0.05990622, 0.05689289, 0.00841569, 0.06298581, 0.28844115])

In [45]:
#наши прогнозы для тестовой выборки
preds_max = logreg_max.predict_proba(X_max_test)[:, 1]
preds_max[:10]

array([0.24470564, 0.00339583, 0.85272663, 0.18579833, 0.04897922,
       0.02163839, 0.07599324, 0.00231151, 0.00400504, 0.28378411])

In [73]:
model = ['mean', 'median', 'max']
precision = []
recall = []
fscore = []
roc_auc = []

In [74]:
#Рассчитаем Precision, Recall, F_score:
precision_mean, recall_mean, thresholds_mean = precision_recall_curve(y_mean_test, preds_mean)
fscore_mean = (2 * precision_mean * recall_mean) / (precision_mean + recall_mean)
# locate the index of the largest f score
ix_mean = np.argmax(fscore_mean)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_mean[ix_mean], 
                                                                        fscore_mean[ix_mean],
                                                                        precision_mean[ix_mean],
                                                                        recall_mean[ix_mean]))
precision.append(precision_mean[ix_mean])
recall.append(recall_mean[ix_mean])
fscore.append(fscore_mean[ix_mean])

Best Threshold=0.239048, F-Score=0.663, Precision=0.569, Recall=0.796


In [75]:
#Рассчитаем Precision, Recall, F_score:
precision_median, recall_median, thresholds_median = precision_recall_curve(y_median_test, preds_median)
fscore_median = (2 * precision_median * recall_median) / (precision_median + recall_median)
# locate the index of the largest f score
ix_median = np.argmax(fscore_median)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_median[ix_median], 
                                                                        fscore_median[ix_median],
                                                                        precision_median[ix_median],
                                                                        recall_median[ix_median]))
precision.append(precision_median[ix_median])
recall.append(recall_median[ix_median])
fscore.append(fscore_median[ix_median])

Best Threshold=0.290133, F-Score=0.728, Precision=0.695, Recall=0.763


In [76]:
#Рассчитаем Precision, Recall, F_score:
precision_max, recall_max, thresholds_max = precision_recall_curve(y_max_test, preds_max)
fscore_max = (2 * precision_max * recall_max) / (precision_max + recall_max)
# locate the index of the largest f score
ix_max = np.argmax(fscore_max)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_max[ix_max], 
                                                                        fscore_max[ix_max],
                                                                        precision_max[ix_max],
                                                                        recall_max[ix_max]))
precision.append(precision_max[ix_max])
recall.append(recall_max[ix_max])
fscore.append(fscore_max[ix_max])

Best Threshold=0.312117, F-Score=0.755, Precision=0.688, Recall=0.837


In [77]:
roc_auc_mean = roc_auc_score(y_mean_test, preds_mean)
print(f'mean: {roc_auc_mean}')
roc_auc.append(roc_auc_mean)

roc_auc_median = roc_auc_score(y_median_test, preds_median)
print(f'median: {roc_auc_median}')
roc_auc.append(roc_auc_median)

roc_auc_max = roc_auc_score(y_max_test, preds_max)
print(f'max: {roc_auc_max}')
roc_auc.append(roc_auc_max)

mean: 0.9492016977731264
median: 0.9620094191522763
max: 0.9692400720972149


In [78]:
results = pd.DataFrame(np.column_stack([model, precision, recall, fscore, roc_auc]), 
                               columns=['model', 'precision', 'recall', 'fscore', 'roc_auc'])

In [79]:
results

,model,precision,recall,fscore,roc_auc
0,mean,0.5685131195335277,0.7959183673469388,0.6632653061224489,0.9492016977731264
1,median,0.6951672862453532,0.763265306122449,0.7276264591439691,0.9620094191522763
2,max,0.6879194630872483,0.8367346938775511,0.7550644567219152,0.9692400720972149


Самый хороший результат показала модель, в которую были поданы максимальные значения векторов. Возможно это связано с тем, что мы берем максимально характеризующие пользователя вектора статей (тем самым "сужая допуски" модели при обучении) из чего она делает более точные прогнозы так как базово обучалась на наиболее подходящих значениях.

In [65]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [ ]:
# #мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
# font = {'size' : 15}

# plt.rc('font', **font)

# cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
# plt.figure(figsize=(10, 8))
# plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
#                       title='Confusion matrix')
# plt.savefig("conf_matrix.png")
# plt.show()